In [35]:
import pandas as pd
import matplotlib as plt
from data import get_train_data


raw_data = get_train_data()

print(f"Number of train groups: {len(raw_data)}")

first_train_key = list(raw_data.keys())[0]
first_train = raw_data[first_train_key][0]

print(f"Keys in first train: {list(first_train.keys())}")
print(f"Station values in first train: {(first_train['stations'][0].keys())}")

Number of train groups: 149
Keys in first train: ['routeName', 'trainNum', 'trainNumRaw', 'trainID', 'lat', 'lon', 'trainTimely', 'iconColor', 'textColor', 'stations', 'heading', 'eventCode', 'eventTZ', 'eventName', 'origCode', 'originTZ', 'origName', 'destCode', 'destTZ', 'destName', 'trainState', 'velocity', 'statusMsg', 'createdAt', 'updatedAt', 'lastValTS', 'objectID', 'provider', 'providerShort', 'onlyOfTrainNum', 'alerts']
Station values in first train: dict_keys(['name', 'code', 'tz', 'bus', 'schArr', 'schDep', 'arr', 'dep', 'arrCmnt', 'depCmnt', 'status', 'stopIconColor', 'platform'])


In [36]:
train_records = []
for train_number, train_list in raw_data.items():
  for train in train_list:
    train_record = {
      'train_number': train_number,
      'route_name': train.get('routeName', ''),
      'train_id': train.get('trainID', ''),
      'lat': train.get('lat'),
      'lon': train.get('lon'),
      'heading': train.get('heading', ''),
      'velocity': train.get('velocity'),
      'train_state': train.get('trainState', ''),
      'status_msg': train.get('statusMsg', ''),
      'origin_code': train.get('origCode', ''),
      'dest_code': train.get('destCode', ''),
      'num_stations': len(train.get('stations', []))
    }
    train_records.append(train_record)

trains_df = pd.DataFrame(train_records)
print("TRAINS DataFrame:")
print(trains_df.head())
print(f"Shape: {trains_df.shape}")

TRAINS DataFrame:
  train_number       route_name train_id        lat         lon heading  \
0            1   Sunset Limited     1-28  30.351255 -103.687212      NW   
1            2   Sunset Limited     2-27  29.767345  -95.366058       E   
2            3  Southwest Chief     3-27  33.872613 -117.769810      SW   
3            3  Southwest Chief     3-28  38.074298 -102.359506       W   
4            4  Southwest Chief     4-27  39.086807  -94.583786       E   

    velocity train_state status_msg origin_code dest_code  num_stations  
0  34.392879      Active                    NOL       LAX            22  
1   0.000000      Active                    LAX       NOL            22  
2  56.550980      Active                    CHI       LAX            32  
3  58.073330      Active                    CHI       LAX            32  
4   0.260976      Active                    LAX       CHI            32  
Shape: (162, 12)


In [37]:
station_records = []
for train_number, train_list in raw_data.items():
  for train in train_list:
    route_name = train.get('routeName', '')
    train_id = train.get('trainID', '')
    
    for station in train.get('stations', []):
      station_record = {
        'train_number': train_number,
        'train_id': train_id,
        'route_name': route_name,
        'station_name': station.get('name', ''),
        'station_code': station.get('code', ''),
        'scheduled_arrival': station.get('schArr'),
        'actual_arrival': station.get('arr'),
        'scheduled_departure': station.get('schDep'),
        'actual_departure': station.get('dep'),
        'status': station.get('status', ''),
        'timezone': station.get('tz', ''),
      }
      station_records.append(station_record)

stations_df = pd.DataFrame(station_records)
print("\nSTATIONS DataFrame:")
print(stations_df.head())
print(f"Shape: {stations_df.shape}")



STATIONS DataFrame:
  train_number train_id      route_name  station_name station_code  \
0            1     1-28  Sunset Limited   New Orleans          NOL   
1            1     1-28  Sunset Limited     Schriever          SCH   
2            1     1-28  Sunset Limited    New Iberia          NIB   
3            1     1-28  Sunset Limited     Lafayette          LFT   
4            1     1-28  Sunset Limited  Lake Charles          LCH   

           scheduled_arrival             actual_arrival  \
0  2025-07-28T09:00:00-05:00  2025-07-28T09:02:00-05:00   
1  2025-07-28T10:30:00-05:00  2025-07-28T10:32:00-05:00   
2  2025-07-28T11:56:00-05:00  2025-07-28T11:55:00-05:00   
3  2025-07-28T12:24:00-05:00  2025-07-28T12:26:00-05:00   
4  2025-07-28T13:55:00-05:00  2025-07-28T13:48:00-05:00   

         scheduled_departure           actual_departure    status  \
0  2025-07-28T09:00:00-05:00  2025-07-28T09:00:00-05:00  Departed   
1  2025-07-28T10:30:00-05:00  2025-07-28T10:33:00-05:00  Departed

In [ ]:
print(f"Number of delays: {stations_df['status'].str.contains('late|delay', case=False, na=False).sum()}")
print(f"Status values: f{stations_df['status'].unique()}")


Number of delays: 0
Status values: f['Departed' 'Station' 'Enroute']


C:\Users\walte\AppData\Local\Temp\ipykernel_18056\3516135735.py:8: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  stations_df['scheduled_departure_dt'] = pd.to_datetime(stations_df['scheduled_departure'])


ValueError: time data "2025-07-29T21:19:02.000Z" doesn't match format "%Y-%m-%dT%H:%M:%S%z", at position 2957. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.